In [1]:
import spacy
import coreferee
from spacy.matcher import Matcher
import pandas as pd
import pickle
from spacy import displacy


In [2]:
nlp = spacy.load("en_core_web_lg")
nlp.max_length = 3200000
nlp.add_pipe('coreferee')


In [3]:
text = ""
with open("./LOTRC_COREF.txt", encoding="utf8") as f:
  try:
    text = f.readlines()
  except Exception as e:
    print(e)


In [4]:
import re
string_text = ""
for line in text:
  string_text += line + "\n"
string_text = re.sub('[\n]+', '\n', string_text)


In [8]:

doc = pickle.load(open("./Dumps/LOTR/LOTRC_COREF_spacy.pkl","rb"))

In [9]:
def get_entities(sent):

  entity1 = ""
  entity2 = ""

  prev_token_dependency = ""    
  prev_token_text = ""   

  prefix = ""
  modifier = "" 
  for tok in nlp(sent):
    if tok.dep_ != "punct":
      if tok.dep_ == "compound":
        if prev_token_dependency == "compound":
          prefix = prev_token_text + " " + tok.text
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        if prev_token_dependency == "compound":
          modifier = prev_token_text + " "+ tok.text
      if tok.dep_.find("subj") == True:
        entity1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prev_token_dependency = ""
        prev_token_text = ""
      if tok.dep_.find("obj") == True:
        entity2 = modifier + " " + prefix + " " + tok.text     
      prev_token_dependency = tok.dep_
      prev_token_text = tok.text
  return [entity1.strip(), entity2.strip()]


In [10]:
def extract_entities_from_sentence(sentence):
  doc = nlp(sentence)
  previous_token_dependency = ""
  previous_token_text = ""

  entita1 = ""
  entita2 = ""

  prefisso = ""
  modificatore = ""
  for token in doc:
    if token.dep_ != "punct":
      if token.dep_ == "compound":
        prefisso = token.text
        if previous_token_dependency == "compound":
          prefisso = previous_token_text + " " + token.text
      if token.dep_.endswith("mod"):
        modificatore = token.text
        if previous_token_dependency == "compound":
          modificatore = previous_token_text + " " + token.text
      if token.dep_.find("subj"):
        entita1 = modificatore+" "+prefisso+" "+token.text
        prefisso=""
        modificatore=""
        previous_token_dependency = ""
        previous_token_text = ""
      if token.dep_.find("obj"):
        entita2 = modificatore+" "+prefisso+" "+token.text
      previous_token_dependency = token.dep_
      previous_token_text = token.text
  return [entita1, entita2]
       

In [11]:
def extract_relations(sentence):
  doc = nlp(sentence)
  matcher = Matcher(nlp.vocab) 
  pattern = [{'DEP': 'ROOT'},
             {'DEP': 'prep', 'OP': "?"},
             {'DEP': 'agent', 'OP': "?"},
             {'POS': 'ADJ', 'OP': "?"}]
  matcher.add("matching_1", [pattern])
  matches = matcher(doc)
  match_index = len(matches)-1
  text = doc[matches[match_index][1]:matches[match_index][2]].text
  return text


In [72]:
def relation_extraction(sentence):
  relations = []
  doc_sentence = nlp(sentence)
  entity1 = ""
  entity2 = ""
  prev_dep = ""
  compound = ""
  rel=""
  #Pattern 1 subject-verb-object
  for token in sentence:
    if token.dep_.find("subj") == True:
      entity1+=" " +token.text
    elif token.dep_.find("obj") == True:
      entity2 += " " + token.text
    elif token.pos_ == "VERB":
      rel+=" "+ token.text

  print(entity1,rel,entity2)

In [36]:
new_sentences[5]

' ever since Bilbo remarkable disappearance and unexpected return '

In [103]:

for sentence in doc.sents:
  sent = nlp(sentence.text)
  # relation_extraction(sentence=sent)
  rel = []
  currentRel = -1
  currentRelText=""
  e1 = ""
  e2 = ""
  for tok in sent:
    if tok.pos_ == "VERB":
      verb_index = tok.i
      already_added = False
      for child in tok.children:
        if "advmod" in child.dep_ or "ccomp" in child.dep_ or "aux" in child.dep_ or "mark" in child.dep_:
          if child.i < verb_index:
            currentRelText += " "+child.text
          else:
            if not already_added:
              currentRelText+=" "+tok.text+" "+child.text
              already_added = True
            else:
              currentRelText+= " "+child.text
        elif "subj" in child.dep_:
          e1+=" "+child.text
        elif "obj" in child.dep_:
          e2+= " "+child.text
      rel.append([e1,currentRelText,e2])
      currentRelText=""


In [105]:
processed_rel = []
print(rel)
for i in range(0,len(rel)):
  current_rel = rel[i]
  if current_rel[0] == '':
    
    next_rel = rel[i+1]
    if current_rel[2] in next_rel[0]:
        if not (current_rel[0] == '' and current_rel[2] == ''):
          processed_rel.append(
              [current_rel[2], current_rel[1]+" "+next_rel[1], next_rel[2]])
          
  elif current_rel[2] == '':
    next_rel = rel[i+1]
   
    if current_rel[0] in next_rel[0]:
      print("ok")
      if not (current_rel[0] == '' and current_rel[2] == ''):

        processed_rel.append(
            [current_rel[0], current_rel[1]+" "+next_rel[1], next_rel[2]])
print(processed_rel)

[]
[]


In [99]:
first = ' Baggins'
second = ' Baggins Baggins'
first in second

True

In [45]:
index = 12
print(get_entities(new_sentences[index]))
extract_relations(new_sentences[index])


['Time', '']
wore


In [22]:
for sent in doc.sents:
  document = nlp(str(sent))
  displacy.render(document, style="dep")
  break

In [20]:
document = nlp(str(doc.sents[0].text))
displacy.serve(document, style="dep")


TypeError: 'generator' object is not subscriptable

In [51]:
triples = []
for sentence in new_sentences:
  try:
    entities = get_entities(sentence)
    relation = extract_relations(sentence)
    if len(entities) ==2 and relation and entities[0] != '' and entities[1] != '':
      triples.append([entities[0], relation, entities[1]])
  except Exception as e:
    pass

In [52]:
triples[:10]

[['Bag Baggins', 'announced', 'special  magnificence'],
 ['Bilbo', 'become', 'back Bilbo travels'],
 ['well  unchanged', 'began', 'mark'],
 ['anyone', 'seemed unfair', 'perpetual  youth'],
 ['far  trouble', 'come', 'Mr'],
 ['Baggins', 'was generous', 'Baggins money'],
 ['most  people', 'were willing', 'Baggins Baggins oddities'],
 ['Baggins', 'remained on', 'visiting Baggins relatives'],
 ['Baggins', 'had', 'close  friends'],
 ['ninety nine ninety Bilbo', 'adopted', 'Bilbo heir']]

In [53]:
len(triples)

7801

In [20]:
sentences[2]

' \n Bilbo was very rich and very peculiar , and had been the wonder of the Shire for sixty years , ever since Bilbo remarkable disappearance and unexpected return '